In [1]:
import os
import xlcalculator


from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from formula_visualiser import FormulaVisualiser


In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_8'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [4]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [5]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [6]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [7]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [8]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [9]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [10]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [11]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [12]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [13]:
series = None
for item in series_iterator:
    if item.formulas != [None, None]:
        series = item
        break


In [14]:
series

Series(series_id=UUID('55021172-7e3e-4c5b-a0e1-d7dd979e4f32'), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Size', formulas=['=INDEX(B:B,MATCH(A2,A:A,0))', '=INDEX(B:B,MATCH(A3,A:A,0))'], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=13, row=2, coordinate='M2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>)

In [15]:
series_mapping = SeriesMapper.map_series(series_dict)

In [16]:
formula_1 = series.formulas[0]
formula_1_ast = FormulaParser.parse_formula(formula_1)

In [17]:
series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)

formula_1_ast_new = series_implementer.replace_range_nodes(formula_1_ast)

INDEX(B:B, MATCH(A2, A:A, 0))
[<RangeNode tvalue: 'B:B', ttype: operand, tsubtype: range>, <FunctionNode tvalue: 'MATCH', ttype: function, tsubtype: >]
[<class 'xlcalculator.ast_nodes.RangeNode'>, <class 'xlcalculator.ast_nodes.FunctionNode'>]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
formula_1_new = SeriesImplementer.serialise_ast_to_formula(formula_1_ast_new)

In [ ]:
formula_1_new

'(SUM(88734fa4c9ea4b8f9548108b3e7bdde5_0_1) + SUM(88734fa4c9ea4b8f9548108b3e7bdde5_0_1))'

In [ ]:
formula_1_ast_new = FormulaParser.parse_formula(f'={formula_1_new}')

In [ ]:
formula_visualiser = FormulaVisualiser()
ast_graph = formula_visualiser.visualise(formula_1_ast_new)
ast_graph.render('formula_1_ast_new')

'formula_1_ast_new.png'